In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json

from urllib.request import urlopen, Request
from urllib.parse import urljoin, urlencode


font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 30}

plt.rc('font', **font)

In [2]:
class Client(object):
    """Encapsulated client. Used for
    """

    @staticmethod
    def get(url, payload=None, auth=None):
        """Performs a GET HTTP request.
        :param url: The url.
        :param payload: The parameters in a dictionary.
        :param auth: A tuple containing username and password.
        """

        if payload is None:
            payload = {}

        if auth is not None:
            payload = merge_two_dicts(payload, auth)
        full_url = '{}?{}'.format(url, urlencode(payload))


        request = Request(full_url)

        try:
            response = urlopen(request)
        except:
            return e.code, None

        return response.code, response.read()

### Getting the results for all matches in a season

In [3]:
from random import randint
from time import sleep

In [4]:
sw_seasons = {
    2018: {1:192, 2: 221, 3: 201, 4: 222},
    2017: {1: 154, 2: 157, 3: 160, 4: 191},
    2016: {1: 126, 2: 129, 3: 132, 4: 135},
    2015: {1: 98, 2: 101, 3: 104, 4: 107},
    2014: {1:25, 2: 28, 3: 31, 4: 34},
}

sm_seasons = {
    2018: {1: 205, 2: 207, 3: 209, 4: 211},
    2017: {1: 167, 2: 169, 3: 171, 4: 173},
    2016: {1: 139, 2: 141, 3: 143, 4: 145},
    2015: {1: 111, 2: 113, 3: 115, 4: 117},
    2014: {1: 68, 2: 73, 3: 76, 4: 79},
}

clubColors = {
    'HHH': '#9c1e14',
    'THH': '#ffffff',
    'H/W': '#fffe54',
    'DUL': '#FF8C00',
    'FUL': '#0a0b0d',
    'BEL': '#9e3231',
    'RAN': '#0a37c4',
    'C/C': '#408559',
    'KEN': '#4babe9',
    'SLH': '#671109',
    'E&E': '#ee4024',
    'REI': '#a7342e',
    'W4H': '#f2e94d',
    'ADV': '#ff00ff',
    'G&G': '#4df297'
}

In [6]:
def getMatchRaceResults(match_id):
    _, data = Client.get(f"https://surreyleague.org/sll/race/{match_id}/json/")
    return json.loads(data.decode('utf-8'))

def matchResultsDataFrame(match_id):
    return pd.DataFrame.from_dict(getMatchRaceResults(match_id)['positions'])

In [7]:
results_list = []
season_json = {}

for s, i in sw_seasons.items():
    season_json[s] = {}
    for m, match_id in i.items():
        season_json[s][m] = getMatchRaceResults(match_id)
        sleep(randint(1,10))

### Now munging the results into the format that I want:

#### Runners per club per season

In [ ]:
totalUniqueRunners = (
    df
    .assign(name=df['givenName'].str.lower() + ' ' + df['familyName'].str.lower())
    .groupby(['season', 'club'])
    .agg({'name': lambda s: s.nunique()})
    .reset_index()
    
)

In [ ]:
uniqueRunners = []
for c in clubColors.keys():
    uniqueRunners.append(
        totalUniqueRunners.loc[lambda d: d['club'] == c]
        .merge(
            pd.DataFrame([[2018, 1], [2017, 1], [2016, 1], [2015, 1], [2014, 1]], columns=['season', 'check']),
            on=['season'], how='right'
        )
        .fillna({'club': c, 'name': 0})
        .drop(columns={'check'})
        .rename(columns={'name': 'count'})
    )
    
totalRunners = pd.concat(uniqueRunners).sort_values(['season', 'club'])

In [ ]:
fig, ax = plt.subplots(figsize=(16,7))  

clubs = totalRunners['club'].drop_duplicates()
margin_bottom = np.zeros(len(totalRunners['season'].drop_duplicates()))
colors = ["#006D2C", "#31A354","#74C476"]

for num, club in enumerate(clubs):
    values = list(totalRunners[totalRunners['club'] == club].loc[:, 'count'])

    totalRunners[totalRunners['club'] == club].plot.bar(x='season',y='count', ax=ax, stacked=True, 
                                                        bottom = margin_bottom, 
                                                        color=clubColors[club], 
                                                        label=club, edgecolor='black')
    margin_bottom += values

plt.title("Number of unique runners per team for Surrey League seasons")
plt.legend(ncol = 3)
plt.show()

### Calculating score per team per season

In [ ]:
season_results = (
    df.loc[lambda d: d['club'].isin(clubColors.keys())]
    .groupby(['season', 'match', 'club'])[['rank']]
    .nth([0, 1, 2, 3, 4])
    .groupby(['season', 'club'])[['rank']].sum()
    .reset_index()
)

In [ ]:
f, ax = plt.subplots(1, figsize=(18, 6))

for c in ['HHH', 'THH', 'C/C', 'FUL', 'RAN', 'BEL', 'SLH', 'KEN', 'H/W']:
    y = season_results.loc[lambda d: d['club'] == c]
    ax.plot(
        y['season'], y['rank'], 
        '-o', markersize='15', markeredgecolor='black',
        label=c, color=clubColors[c])
plt.show()

H/W Seasons:

In [ ]:
hwSeasons = df.loc[lambda d: (d['club'] == 'H/W')].reset_index(drop=True)
hwSeasons = (
    hwSeasons
    .join(hwSeasons.groupby(['season', 'match'])[['rank']].rank(ascending=True).rename(columns={'rank':'teamPosition'}))
    .assign(name=lambda d: d['givenName'].str.lower() + ' ' + d['familyName'].str.lower())
)

In [ ]:
fig, ax = plt.subplots(figsize=(14,7)) 

matches = hwSeasons['match'].drop_duplicates()
margin_bottom = np.zeros(len(hwSeasons['season'].drop_duplicates()))
colors = ["#900C3F", "#C70039", "#FF5733", "#FFC300"]

for num, match in enumerate(matches):
    matchResult = hwSeasons[(hwSeasons['match'] == match) & (hwSeasons['teamPosition'] <= 5)].groupby(['season'])[['rank']].sum().reset_index()
    values = list(matchResult['rank'])

    matchResult.plot.bar(x='season',y='rank', ax=ax,
                         stacked=True, 
                         bottom=margin_bottom, 
                         color=colors[num], label=f"Match {match}")
    margin_bottom += values

plt.title("Total season score (split into matches) for HW throughout the seasons")
plt.ylabel("Match Score")
plt.show()

In [ ]:
f, ax = plt.subplots(1, figsize=(8, 16))

seasonAverages = (
    hwSeasons.loc[lambda d: d['name'].isin(hwSeasons[hwSeasons['season'] == 2018]['name'].unique())]
    .groupby(['name', 'season']).agg({'rank': 'mean', 'perf': 'count'})
    .unstack()['rank']
    .sort_values(2018, ascending=True)
)

colors = ['C0', 'C1', 'C2', 'C3', 'C4']

for s, c in zip([2014, 2015, 2016, 2017, 2018], colors):
    plt.plot(seasonAverages[s], seasonAverages.index, '.', markersize=15, color=c, label=f"season {s}")
plt.legend()
plt.show()

Unique H/W runners in the season:

In [ ]:
thisSeason['name'].unique()

# A team scorers

In [ ]:
lastSeason.loc[lambda d: d['teamPosition'] <= 5]['name'].unique()

In [ ]:
thisSeason.loc[lambda d: d[]]

# B team scorers

In [ ]:
thisSeason.loc[lambda d: (d['teamPosition'] > 5) & (d['teamPosition'] <= 10)]['name'].unique()

In [ ]:
(
    thisSeason
    .groupby(['name'])
    .agg({'perf': 'count'})
    .sort_values('perf', ascending=False)
)

### SEAA Road Relayds 2019

In [ ]:
def get_seconds(t):
    time = [int(i) for i in t.split(':')]
    return time[0] * 60 + time[1]

In [ ]:
dataLong = pd.read_csv("/Users/eileentoomer/Downloads/seaaLongLegs2019.csv")
dataShort = pd.read_csv("/Users/eileentoomer/Downloads/seaaShortLegs2019.csv")

plt.style.use('seaborn')

In [ ]:
data = (
    pd.concat([dataLong.assign(long=True), dataShort.assign(long=False)])
    .assign(seconds=lambda d: d['time'].apply(get_seconds))
    .reset_index(drop=True)
)

In [ ]:
teamRank = (
    data.groupby('team')['seconds']
    .agg([np.sum, 'count'])
    .loc[lambda d: d['count'] == 12]
    .rank(ascending=True)
    .rename(columns={'sum': 'rank'})['rank']
)

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(16, 6), sharey=True)
for i, a in zip([True, False], ax):
    a.hist(data.loc[lambda d: d['long'] == i]['seconds'] / 60, bins=25, color='C3')
    title = 'Time Long Legs' if i else 'Time Short Legs'
    a.set(title=title, xlabel='Time (mins)', ylabel='Number of runners')
    
for i, row in data.loc[lambda d: d['team'].str.contains('Hercules Wimbledon')].iterrows():
    c = 'red' if row['team'] == 'Hercules Wimbledon AC "A"' else 'yellow'
    alpha = 1 if row['team'] == 'Hercules Wimbledon AC "A"' else 0
    if row['long']:
        ax[0].axvline(row['seconds'] / 60, color=c, alpha=alpha)
    else:
        ax[1].axvline(row['seconds'] / 60, color=c, alpha=alpha)    
plt.show()

In [ ]:
grouped_data = (
    data
    .groupby(['long', 'team'])['seconds']
    .agg([pd.np.min, pd.np.max, pd.np.mean])
    .join(teamRank)
    .loc[lambda d: ~d['rank'].isnull()]
    .sort_values('rank', ascending=False)
)

In [ ]:
def plotMeanMinMaxOfTeams(ax, data):
    for i, row in data.iterrows():
        c=f'C{i % 10}'
        x = [row['amin'] / 60, row['amax'] / 60]
        y = [row['team']] * 2
        ax.plot(x, y, 'o-', markersize=5, color=c)
        ax.plot(row['mean'] / 60, i, 's', markersize=10, color=c)
    return ax

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(12, 12), sharey=True)

plotMeanMinMaxOfTeams(ax[0], grouped_data.loc[True].loc[lambda d: d['rank'] <= 40].reset_index())
ax[0].set(title='Long legs min/mean/max time (mins)')
plotMeanMinMaxOfTeams(ax[1], grouped_data.loc[False].loc[lambda d: d['rank'] <= 40].reset_index())
ax[1].set(title='Short legs min/mean/max time (mins)')
plt.show()

In [ ]:
clubs = ['Belgrave Harriers', 'Bedford & County AC "A"', 'Hercules Wimbledon AC "A"', 'Highgate Harriers "A"']

### Testing all figures

In [ ]:
import gviz_api
import pandas as pd
import numpy as np

In [ ]:
def get_minutes(t):
    """
    Function turns string time to number of minutes
    :param t: String in time format mm:ss
    :return: Time as number of seconds
    """
    time = [int(i) for i in t.split(':')]
    return time[0] + time[1] / 60


def getAllData():
    dataLong = pd.read_csv("/Users/eileentoomer/Downloads/womenSEAALongLegs.csv")
    dataShort = pd.read_csv("/Users/eileentoomer/Downloads/womenSEAAShortLegs.csv")

    return (
        pd.concat([dataLong.assign(long=True), dataShort.assign(long=False)])
        .assign(minutes=lambda d: d['time'].apply(get_minutes))
        .assign(team=lambda d: d['team'].pipe(stripSpeechMarksFromClubs))
        .reset_index(drop=True)
    )


def getTeamRanksFromRace(data, count):
    """
    Returns the ranking positions of all teams that finished (i.e. had 12 runners)
    :param data: Dataframe containing team/seconds
    :return:
    """
    return (
        data.groupby('team')['minutes']
        .agg([np.sum, 'count'])
        .loc[lambda d: d['count'] == count]
        .rank(ascending=False)
        .rename(columns={'sum': 'rank'})[['rank']]
        .reset_index()
    )


def teamRankToDict(teamRanks):
    return pd.Series(teamRanks['rank'].values, index=teamRanks['team']).to_dict()


def getRunnerLegRanking(data):
    return data.groupby(['long'])['minutes'].rank(ascending=True).rename('legRank')

def stripSpeechMarksFromClubs(clubs):
    return clubs.str.replace('"', '')


def getTeamResults():
    data = getAllData()
    teams = getTeamRanksFromRace(data, 6)
    runners = getRunnerLegRanking(data)
    return data.join(runners).merge(teams, on='team')

In [ ]:
def getResultsStringByLegLength(df, long=True):
    jsData = []

    for c in df['team'].unique():
        rename_dict = {'rank': c, "name": f"name {c}"}
        jsData.append(
            df.assign(name=lambda d: d['name'] + " (" + d['time'].str[:5] + ")")
            .loc[lambda d: (d['team'] == c) & (d['long'] == long)][['minutes', 'rank', 'name']].rename(columns=rename_dict))
        
    return pd.concat(jsData, sort=False)


def generateResultsStringForLegs(data, long):
    jsDataFrame = getResultsStringByLegLength(df, long)
    cols = [(i, "string") if "name" in i else (i, 'number') for i in list(jsDataFrame)]
    
    data_table = gviz_api.DataTable(cols)
    data_table.LoadData(list(jsDataFrame.itertuples(index=False, name=None)))
    
    return data_table.ToJSon().replace('NaN', 'null').replace('"nan"', 'null')


def getTeamRankDictionary(teamRank):
    teamRankArrayDict = []
    for i, row in teamRank.iterrows():
        t = row['team']
        teamRankArrayDict.append({"v": row['rank'], "f": f'{t}'})
    return teamRankArrayDict

In [ ]:
df = getTeamResults()

In [ ]:
longLegs = generateResultsStringForLegs(df, True)
shortLegs = generateResultsStringForLegs(df, False)

In [ ]:
stackedBoxChart = (
    df[['team', 'leg', 'minutes', 'name', 'rank']]
    .groupby(['team', 'leg'])
    .agg({'minutes': 'mean', 'name': 'max', 'rank': 'max'})
    .unstack('leg')
    .sort_values(('rank', 1), ascending=False)
)

In [ ]:
resultsData = []
cols = []

for i, row in stackedBoxChart.iterrows():
    l = [i]
    for j in range(6):
        l.append(row['minutes'][j + 1])
        # l.append(row['name'][j + 1])
    resultsData.append(tuple(l))
    

In [ ]:
cols = [('team', 'string')]
for i in range(6):
    cols.append((f"Time leg {i + 1}", "number"))
    #cols.append(("runner", "string", "role : tooltip"))

In [ ]:
data_table = gviz_api.DataTable(cols)
data_table.LoadData(list(resultsData))

In [ ]:
data_table.ToJSon()

In [ ]:
c = ['Genre', 'Fantasy & Sci Fi', 'Romance', 'Mystery/Crime', 'General', 'Western', 'Literature']

col = [(i, 'number') if i != 'Genre' else (i, 'string') for i in c]

rows = [('2010', 10, 24, 20, 32, 18, 5), ('2020', 16, 22, 23, 30, 16, 9), ('2030', 28, 19, 29, 30, 12, 13)]
      

In [ ]:
data_table_test = gviz_api.DataTable(cols)
data_table_test.LoadData(rows)

In [ ]:
data_table_test.ToJSon()